## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [24]:
attrition_df['Department'].value_counts()

,count
Department,
Research & Development,961
Sales,446
Human Resources,63


In [23]:
attrition_df['Attrition'].value_counts()

,count
Attrition,
No,1233
Yes,237


In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [4]:
# Create a list of at least 10 column names to use as X data
X_list = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[X_list]


# Show the data types for X_df
X_df.dtypes



,0
Education,int64
Age,int64
DistanceFromHome,int64
JobSatisfaction,int64
OverTime,object
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64


In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)


In [6]:
X_train['OverTime']

,OverTime
464,No
512,No
480,No
680,No
1023,No
...,...
715,Yes
905,No
1096,Yes
235,Yes


In [7]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train['OverTime'] = X_train['OverTime'].map({'Yes': 1, 'No': 0})
X_test['OverTime'] = X_test['OverTime'].map({'Yes': 1, 'No': 0})

In [8]:
X_train['OverTime'].value_counts()


,count
OverTime,
0,798
1,304


In [9]:
# Create a StandardScaler
standard_scaler = StandardScaler()

# Fit the StandardScaler to the training data
standard_scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = standard_scaler.transform(X_train)
X_test_scaled = standard_scaler.transform(X_test)


In [10]:
from sklearn.preprocessing import OneHotEncoder
# Create a OneHotEncoder for the Department column
dept_oh_encoder = OneHotEncoder()

# Fit the encoder to the training data
dept_oh_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_dept = dept_oh_encoder.transform(y_train[['Department']])
y_test_dept = dept_oh_encoder.transform(y_test[['Department']])



In [11]:
# Create a OneHotEncoder for the Attrition column
att_oh_encoder = OneHotEncoder()

# Fit the encoder to the training data
att_oh_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_att = att_oh_encoder.transform(y_train[['Attrition']])
y_test_att = att_oh_encoder.transform(y_test[['Attrition']])


## Create, Compile, and Train the Model

In [12]:
# Find the number of columns in the X training data
X_train.shape[1]

# Create the input layer
input = layers.Input(shape=(X_train.shape[1],))

# Create at least two shared layers

shared_layer_1 = layers.Dense(64, activation='relu', name='shared_layer_1')(input)
shared_layer_2 = layers.Dense(128, activation='relu', name='shared_layer_2')(shared_layer_1)


In [13]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(32, activation='relu', name='department_hidden')(shared_layer_2)

# Create the output layer categorical_crossentropy
department_output = layers.Dense(3, activation='softmax', name='department_output')(department_hidden)


In [14]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu', name='attrition_hidden')(shared_layer_2)

# Create the output layer binary_crossentropy
attrition_output = layers.Dense(2, activation='sigmoid', name='attrition_output')(attrition_hidden)


In [15]:
# Create the model
model = Model(inputs=input, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy','attrition_output':"binary_crossentropy"},
               metrics={'department_output': 'accuracy','attrition_output':"accuracy"})

# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 10)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared_layer_1 (Dense)    │ (None, 64)             │            704 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared_layer_2 (Dense)    │ (None, 128)            │          8,320 │ shared_layer_1[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_hidden (Dense) │ (None, 32)             │          4,128 │ shared_layer_2[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_hidden (Dense)  │ (None, 32)             │          4,128 │ shared_layer_2[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             99 │ department_hidden[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             66 │ attrition_hidden[0][0] │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
y_train_att

<1102x2 sparse matrix of type '<class 'numpy.float64'>'
	with 1102 stored elements in Compressed Sparse Row format>

In [17]:
# Train the model .toarray()

# , validation_data=(X_test_scaled, [y_test_dept.toarray(), y_test_att.toarray()]))

model.fit(X_train_scaled,
 [ y_train_dept.toarray(),  y_train_att.toarray()],
          #validation_data=(X_test_scaled, [y_test_dept.toarray(), y_test_att.toarray()]),
          #validation_split=0.2
          batch_size=32,
          epochs=100)

Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - attrition_output_accuracy: 0.6847 - attrition_output_loss: 0.5807 - department_output_accuracy: 0.6561 - department_output_loss: 0.5719 - loss: 1.1528
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - attrition_output_accuracy: 0.8436 - attrition_output_loss: 0.4140 - department_output_accuracy: 0.6569 - department_output_loss: 0.4852 - loss: 0.8993
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - attrition_output_accuracy: 0.8610 - attrition_output_loss: 0.3406 - department_output_accuracy: 0.6600 - department_output_loss: 0.4724 - loss: 0.8130
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8638 - attrition_output_loss: 0.3562 - department_output_accuracy: 0.6565 - department_output_loss: 0.4773 - loss: 0.8335
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8846 - attrition_output_loss: 0.3094 - department_output_accuracy: 0.6451 - department_output_loss: 0.

In [18]:
# Evaluate the model with the testing data
model_eval=model.evaluate(X_test_scaled, [y_test_dept.toarray(), y_test_att.toarray()])

# Print the results
print(model_eval)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.7603 - attrition_output_loss: 2.2466 - department_output_accuracy: 0.5442 - department_output_loss: 1.4830 - loss: 3.7393  
[3.363168716430664, 1.449210286140442, 1.850812554359436, 0.7853260636329651, 0.5461956262588501]


In [21]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {model_eval[4]}")
print(f"Attrition Accuracy: {model_eval[3]}")

Department Accuracy: 0.5461956262588501
Attrition Accuracy: 0.7853260636329651


In [22]:
#Department predictions accuracy: 0.5271739363670349
#Attrition predictions accuracy: 0.8260869383811951

# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. The accuracy metric is not the best metric to use on this data because the y data are significantly imbalanced, and other metrics, like Precision, Recall, and F1 Score, are more suited for imbalanced datasets.  
2.  I used the sigmoid activation function on the attrition output layer and the softmax activation function on the department output layer because the attrition output was from 0 to 1  (yes or no). In contrast, the department output was split between three options.
3. A few ways to improve the model would be to use a larger dataset, change the metric from accuracy to Precision, Recall, and/or F1 score if the dataset is still imbalanced, or changing the training parameters of the model (i.e. batch_size, validation_split, epochs, etc).